In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
for cm in ChapterMark.objects.all():
    print(cm.audio)

1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
1 - foo audio
2 - asdf
2 - asdf
2 - asdf


<IPython.core.display.Javascript object>

In [56]:
for audio in Audio.objects.all():
    print(audio.pk, audio)

1 1 - foo audio
2 2 - asdf
3 3 - next audio
4 4 - foobar
14 14 - audio form test


<IPython.core.display.Javascript object>

In [57]:
Audio.objects.get(pk=14).delete()

(4, {'cast.ChapterMark': 3, 'cast.Audio': 1})

<IPython.core.display.Javascript object>

# Trying to make the audio form work

In [58]:
from django import forms

from wagtail.admin.forms.collections import BaseCollectionMemberForm

from cast.wagtail_forms import FakePermissionPolicy

<IPython.core.display.Javascript object>

In [59]:
class ChapterMarkForm(forms.ModelForm):
    class Meta:
        model = ChapterMark
        fields = ("start", "title", "link", "image")


class ChapterMarksField(forms.CharField):
    def to_python(self, value):
        chaptermarks = []
        for line in value.split("\n"):
            splitted = line.split()
            if len(splitted) < 2:
                continue
            start, *parts = splitted
            title = " ".join(parts)
            form = ChapterMarkForm({"start": start, "title": title})
            if form.is_valid():
                chaptermarks.append(form.save(commit=False))
        print("chaptermarks: ", chaptermarks)
        return chaptermarks


class AudioForm(BaseCollectionMemberForm):
    chaptermarks = ChapterMarksField(widget=forms.Textarea, required=False)
    permission_policy = FakePermissionPolicy()

    class Meta:
        model = Audio
        fields = [
            "title",
            "subtitle",
            "m4a",
            "mp3",
            "oga",
            "opus",
            "tags",
            "collection",
        ]

    def save_chaptermarks(self, audio):
        chaptermarks = self.cleaned_data.get("chaptermarks", [])
        for cm in chaptermarks:
            cm.audio = audio
            cm.save()

    def save(self, commit=True):
        audio = super().save(commit=commit)
        if commit:
            self.save_chaptermarks(audio)
        return audio

<IPython.core.display.Javascript object>

In [60]:
user = User.objects.first()
chaptermarks_text = """
00:00:28.433 News aus der Szene
00:02:40.964 Packaging
00:12:24.409 Dokumentation
"""
# chaptermarks = chaptermarks_text_to_rows(chaptermarks_text)
data = {
    "title": "audio form test",
    "subtitle": "subtitle for audio test",
    "chaptermarks": chaptermarks_text,
}
audio = Audio(user=user)
form = AudioForm(data, instance=audio)

<IPython.core.display.Javascript object>

In [61]:
form.is_valid()

chaptermarks:  [<ChapterMark: None 00:00:28.433000 News aus der Szene>, <ChapterMark: None 00:02:40.964000 Packaging>, <ChapterMark: None 00:12:24.409000 Dokumentation>]


True

<IPython.core.display.Javascript object>

In [62]:
form.cleaned_data

{'title': 'audio form test',
 'subtitle': 'subtitle for audio test',
 'm4a': None,
 'mp3': None,
 'oga': None,
 'opus': None,
 'tags': [],
 'chaptermarks': [<ChapterMark: None 00:00:28.433000 News aus der Szene>,
  <ChapterMark: None 00:02:40.964000 Packaging>,
  <ChapterMark: None 00:12:24.409000 Dokumentation>]}

<IPython.core.display.Javascript object>

In [63]:
audio = form.save()

<IPython.core.display.Javascript object>

In [64]:
audio

<Audio: 15 - audio form test>

<IPython.core.display.Javascript object>

In [65]:
for cm in audio.chaptermarks.all():
    print(cm)

25 00:00:28.433000 News aus der Szene
26 00:02:40.964000 Packaging
27 00:12:24.409000 Dokumentation


<IPython.core.display.Javascript object>